In [1]:
import tensorflow as tf

# Load the model
model = tf.keras.models.load_model('C:\\Users\\bilal\\Desktop\\food_v5\\main\\output_model.h5')

# Print model summary
model.summary()

# Print input details
print("\nInput details:")
for layer in model.layers:
    if hasattr(layer, 'input'):
        print(f"Layer {layer.name} input: {layer.input}")

# Print output details
print("\nOutput details:")
for layer in model.layers:
    if hasattr(layer, 'output'):
        print(f"Layer {layer.name} output: {layer.output}")

Model: "flat_efficientnetB1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 augmentation_layers (Funct  (None, 224, 224, 3)          0         ['input_1[0][0]']             
 ional)                                                                                           
                                                                                                  
 preprocessing_layers (Func  (None, 224, 224, 3)          0         ['augmentation_layers[0][0]'] 
 tional)                                                                                          
                                                                                

In [2]:
import json
import numpy as np

In [12]:
def preprocess_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Expand dimensions to create batch of size 1
    img_array = tf.reshape(img_array, [-1, 224, 224, 3])
    return img_array


# Function to perform inference on an image
def predict_image(image_path, model):
    img = preprocess_image(image_path)
    outputs = model.predict(img)

    category_output = decode_category_prediction(outputs[0])
    ingredients_output = decode_ingredients_prediction(outputs[1])
    calorie_output = decode_single_value_prediction(outputs[2])
    carbs_output = decode_single_value_prediction(outputs[3])
    protein_output = decode_single_value_prediction(outputs[4])
    fat_output = decode_single_value_prediction(outputs[5])

    return {
        "category": category_output,
        "ingredients": ingredients_output,
        "calorie": calorie_output,
        "carbs": carbs_output,
        "protein": protein_output,
        "fat": fat_output
    }


# Function to decode category prediction
def decode_category_prediction(prediction):
    # Load encoded categories from JSON file
    with open('C:\\Users\\bilal\\Desktop\\food_v5\\main\\encoded_food_categories.json', 'r') as f:
        encoded_categories = json.load(f)
    prediction = tf.reshape(prediction, [-1])
    index = tf.argmax(prediction)
    index = index.numpy()  # Convert TensorFlow tensor to NumPy array
    # Get category label corresponding to the index
    categories = list(encoded_categories.keys())  # List of category labels
    decoded_category = categories[index].replace('_', ' ')
    return decoded_category


# Function to decode ingredients prediction
def decode_ingredients_prediction(prediction):
    with open('D:\\food_v5\\main\\encoded_ingredients.json', 'r') as f:
        encoded_ingredients = json.load(f)
    prediction = tf.reshape(prediction, [-1])
    values, indices = tf.math.top_k(prediction, k=10, sorted=True)
    indices = indices.numpy()  # Convert TensorFlow tensor to NumPy array
    values = values.numpy()    # Convert TensorFlow tensor to NumPy array
    ingredients = list(encoded_ingredients.keys())
    result = []
    for i in range(len(values)):
        if values[i] >= 0.5:  # Adjust threshold as needed
            result.append(ingredients[indices[i]])
        else:
            break
    return result

# Function to decode single value predictions (like calorie, carbs, protein, fat)
def decode_single_value_prediction(prediction):
    result_tensor = tf.reshape(prediction, [-1])
    value = result_tensor.numpy()
    return abs(value[0])

# Function to calculate mass based on ingredient densities and volume
def calculate_mass(ingredients_list, volume):
    with open('D:\\food_v5\\main\\encoded_ingredients_density.json', 'r') as f:
        encoded_ingredients_density = json.load(f)
    
    total_density = 0.0
    for ingredient in ingredients_list:
        if ingredient in encoded_ingredients_density:
            density = encoded_ingredients_density[ingredient]
            total_density += density
            #print(total_density)

    total_density = total_density * 0.05
    mass = total_density * volume
    return mass

# usage:
image_path = 'D:\\food_v6\\image2.jpg'
prediction = predict_image(image_path, model)
print("Prediction:")
print(prediction)
##############################################################
# Read volume from text file
with open('temp_val.txt', 'r') as f:
    volume = float(f.read().strip())
    print(f'Estimated food volume: {volume} Liter')

# Calculate mass based on ingredients and volume
ingredients_list = prediction['ingredients']
mass = calculate_mass(ingredients_list, volume)
print(f"Estimated Mass: {mass} grams")

# Calculate the final Nutrition facts estimations
total_calories = mass * prediction['calorie']
total_fat =  mass * prediction['fat']
total_carbs = mass * prediction['carbs']
total_protein = mass * prediction['protein']
print(f"Total calories: {total_calories}")
print(f"Total fat: {total_fat}")
print(f"Total carbs: {total_carbs}")
print(f"Total protein: {total_protein}")

1/1 [==============================] - 0s 67ms/step
Prediction:
{'category': 'pizza', 'ingredients': ['cheese', 'oil', 'pizza', 'pepper'], 'calorie': 3.6140056, 'carbs': 0.13980232, 'protein': 0.09922809, 'fat': 0.28153133}
Estimated food volume: 2.6083359877972097 Liter
Estimated Mass: 342.99618239533305 grams
Total calories: 1239.5901121711418
Total fat: 96.56417276037129
Total carbs: 47.95166266453944
Total protein: 34.034856583892335
